In [11]:
import pandas as pd
df_compdata_ref_auth = pd.read_csv('../data/compdata_ref_author.csv')

journal_IFs = pd.read_csv('../data/Impact_factor_2020.csv', index_col=False)

## Match journal names

In [128]:
journal_query = list(set(df_compdata_ref_auth.journal.dropna()))
journal_choices = journal_IFs['Full Journal Title'].dropna().tolist()

In [140]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

limit = 5

journal_query_result = dict()
for q in journal_query:
    q_ret = process.extract(q, journal_choices, limit=limit) 
    journal_query_result[q] =  [i[0] for i in q_ret]

In [141]:
print(len(journal_query_result))
f = open("Query2Journal.csv", "w")
for name in journal_query_result.keys():
    print(str(name) + '\t'+ '\t'.join(journal_query_result[name]), file=f)

f.close()

1232


## Retrieve impact factors of journals

In [197]:
# load manually curated results

journal2imf = dict()
name2journal = dict()

f = open("journal.txt", "r")
for lcon in f:
    line =lcon.split('\t')
    if len(line) == 3 and line[2].rstrip()!='':
        journal2imf[line[1]] = line[2].rstrip()
    if len(line) >= 2 and line[1].rstrip()!='':
        name2journal[line[0]] = line[1]

f.close()
print(len(journal2imf))
print(len(name2journal))

33
612


In [198]:
# load impact factors of journals
for idx in journal_IFs.index:
    if journal_IFs['Full Journal Title'].loc[idx] in journal2imf.keys():
        continue
    journal2imf[journal_IFs['Full Journal Title'].loc[idx]] = journal_IFs['Journal Impact Factor'].loc[idx]

print(len(journal2imf))

# remove journals without impact factors
name2journal = {k: v for k, v in name2journal.items() if v in journal2imf.keys()}

12198


In [221]:
# map impact factors to papers
import numpy as np

paper_names = df_compdata_ref_auth['journal'].tolist()
impact_f = [float(journal2imf[name2journal[x]]) if x in name2journal.keys() else np.nan for x in paper_names ]

len(impact_f)
df_compdata_ref_auth['impact_f'] = impact_f

In [222]:
# Replace null values with minimum values
# df_compdata_ref_auth['impact_f'] = df_compdata_ref_auth.apply(lambda row: df_compdata_ref_auth['impact_f'].min() if np.isnan(row['impact_f']) and row['collection']=='pubmed' else row['impact_f'],axis=1)

df_compdata_ref_auth[['impact_f','collection']]

,impact_f,collection
0,2.429,pubmed
1,4.981,pubmed
2,6.551,pubmed
3,4.074,pubmed
4,4.986,pubmed
...,...,...
5753,NaN,biorxiv
5754,NaN,biorxiv
5755,NaN,biorxiv
5756,NaN,biorxiv


In [223]:
df_compdata_ref_auth.columns

Index(['Unnamed: 0', 'title', 'abstract', 'journal', 'DOI', 'date',
       'collection', 'published', 'nauthors', 'topic.Healthcare',
       'topic.Drug discovery', 'topic.Epidemiology', 'topic.Clinics',
       'topic.Chest X-Ray', 'topic.Genomics', 'topic', 'numcit', 'influcit',
       'DL_Pdf', 'DL_Full', 'DL_Abstract', 'ref', 'authorsId', 'authorsNames',
       'p_doi', 'p_journal', 'impact_f'],
      dtype='object')

In [224]:
df_compdata_ref_auth.collection.head(3)

0    pubmed
1    pubmed
2    pubmed
Name: collection, dtype: object

In [239]:
df_compdata_ref_auth_redup = df_compdata_ref_auth.drop_duplicates(subset=['title'])
len(df_compdata_ref_auth_redup)

5686

In [240]:
df_compdata_ref_auth_redup['impact_f'].to_csv('impact_f.csv')

In [241]:
df_compdata_ref_auth_redup.groupby('collection').count()

,Unnamed: 0,title,abstract,journal,DOI,date,published,nauthors,topic.Healthcare,topic.Drug discovery,...,influcit,DL_Pdf,DL_Full,DL_Abstract,ref,authorsId,authorsNames,p_doi,p_journal,impact_f
collection,,,,,,,,,,,,,,,,,,,,,
arxiv,758,758,758,38,758,758,38,758,758,758,...,758,0,0,0,740,752,752,200,495,21
biorxiv,623,623,623,0,623,623,95,623,623,623,...,623,592,592,592,606,604,604,611,146,0
medrxiv,1247,1247,1247,0,1247,1247,138,1247,1247,1247,...,1247,1193,1193,1193,1201,1238,1239,1247,1215,0
pubmed,3058,3057,3058,3058,3058,3058,0,3058,3055,3055,...,3058,0,0,0,2621,3049,3049,3013,3055,1917


In [242]:
df_compdata_ref_auth_redup['impact_f'].count()

1938

In [244]:
df_compdata_ref_auth_redup['impact_f'].size

5686